In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization
from matplotlib import pyplot as plt # data visualization
import time
import re
import pickle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
cdf = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv")
cdf_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv")

In [ ]:
cdf

# **1. Knowing the Data:~**

In [ ]:
cdf.set_index('id', inplace=True)
cdf_test.set_index('id', inplace=True)

In [ ]:
display(cdf.columns)
display(cdf_test.columns)

In [ ]:
cdf.rename(columns={"ext_col":"ext_color", "int_col":"int_color", "milage":"mileage"}, inplace=True)
cdf_test.rename(columns={"ext_col":"ext_color", "int_col":"int_color", "milage":"mileage"}, inplace=True)

In [ ]:
display(cdf.sample(5))
display(cdf_test.sample(5))

In [ ]:
print(f"Shape of training data: {cdf.shape}, \nShape of test data: {cdf_test.shape}")

In [ ]:
cdf.isnull().sum()

In [ ]:
cdf_test.isnull().sum()

In [ ]:
display(cdf[cdf.duplicated()])
display(cdf_test[cdf_test.duplicated()])

### **SUMMARY from above //---**
* The train dataset has 13 columns, with "id" column set as index. So effectively 12. The test data has 11.
* There are **188533 rows in the train data.** While the **test dataset has 1.25lac rows.** 
* **No duplicates** in either of datset. **Both has three columns that has some null values.**
* "price" is the target column. Have **changed ext_col & int_col column names to ext_color & int_colour,** meaning Exterior & Interior Colour.

## **1.1. Details of the Numerical Columns:**

In [ ]:
cdf.describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 3), sharey=True)
fig.suptitle('Box Plots of --')

# mileage
sns.boxplot(ax=axes[0], x=cdf["mileage"])
axes[0].set_title("Mileage of Cars")

# price
sns.boxplot(ax=axes[1], x=cdf["price"])
axes[1].set_title("Price of Cars")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

# mileage
sns.histplot(x=cdf["mileage"],ax=axes[0],kde=True, bins=25)
axes[0].set_title("Mileage of Cars")

# price
sns.histplot(x=cdf["price"],ax=axes[1], bins=50)
axes[1].set_title("Price of Cars")

In [ ]:
cdf.model_year.value_counts().head(5)

In [ ]:
sns.set(rc={'figure.figsize':(12,4)})
yvc = cdf.model_year.value_counts().sort_index()
yr = sns.lineplot(x=list(yvc.index), y=yvc)
plt.xticks(list(yvc.index)[::2], rotation=90)

for i in range(0,len(yvc),2):
    yr.vlines(x=yvc.index[i],ymin=0,ymax=yvc.values[i],colors='grey',)
    yr.text(yvc.index[i],yvc.values[i], f"{yvc.values[i]}",ha='center',va="top", fontweight='bold', fontsize=9)

plt.title("No. of cars per model_year")
yr.set_yticklabels([])
plt.show()

## **1.2. Details of the Categorical Columns:**

In [ ]:
cdf.describe(include="object")

In [ ]:
cdf.accident.value_counts().plot(kind="pie", title="Reports for Accident", ylabel="",
                                y=cdf.accident.value_counts(), figsize=(3,3),
                                autopct="%1.01f%%")
plt.show()

##### Let's make a little change to replace these long values "At least 1 accident or damage reported" & "None reported"

In [ ]:
cdf.rename(columns={"accident":"accident_reported"}, inplace=True)
cdf_test.rename(columns={"accident":"accident_reported"}, inplace=True)
cdf.accident_reported = np.where(cdf.accident_reported=="None reported","No","Yes")
cdf_test.accident_reported = np.where(cdf_test.accident_reported=="None reported","No","Yes")

In [ ]:
cdf.sample(3)

In [ ]:
cdf.fuel_type.value_counts()

In [ ]:
cdf.engine.value_counts(normalize=True)*100

In [ ]:
cdf.transmission.value_counts(normalize=True)*100

In [ ]:
display(cdf.ext_color.value_counts())
display(cdf.int_color.value_counts())

In [ ]:
display(cdf.clean_title.value_counts())
cdf.clean_title.isnull().sum()

### **SUMMARY from above //---**
* **Mileage column is right skewed** as it generally should be. **Price column has a lot of outliers,** probably fo the presence of a few supar cars.
* There are 12 cars of model year 1974. Most cars are of years between 2015 & 2022.
* **23% cars have at least one accident or damage being reported.**
* **`engine` & `transmission` columns have so many unique values.** Some column transformation can be done on those columns like creating a few smaller columns like Cylinder No, or Capacity etc.

# **Data Cleaning and Processing:~**

In [ ]:
cdf.isnull().sum()

In [ ]:
cdf_test.fuel_type.unique()

In [ ]:
cdf['fuel_type'] = cdf['fuel_type'].replace(['–', 'not supported'], 'None')
cdf_test['fuel_type'] = cdf_test['fuel_type'].replace(['–', 'not supported'], 'None')

In [ ]:
cdf.transmission.unique()

In [ ]:
def extract_gear_and_txtype(transmission_info):
    pattern = re.search(r'(\d{1,2}[\s-]?speed)?\s*(Electronically Controlled Automatic|Automatic|At\/Mt|A\/T|AT|M\/T|CVT|Manual|Variable|Transmission Overdrive|Fixed|DCT|Mt|Transmission w/Dual Shift Mode)?\s*',transmission_info,re.IGNORECASE)
    
    gear = pattern.group(1) if pattern.group(1) else None
    txtype = pattern.group(2) if pattern.group(2) else "Other"
    return gear, txtype

gear = []
transmission_type = []
for tx in cdf.transmission:
    ngear = extract_gear_and_txtype(tx)[0]
    if ngear!=None:
        ngear = int(ngear.split("-")[0].split(" ")[0]) #to tackle both 6-speed & 6 speed

    gear.append(ngear)
    
    txtype = extract_gear_and_txtype(tx)[1]
    if txtype!=None:
        if txtype=="At/Mt": txtype="AMT"
        elif txtype.lower() in ['a/t','at','transmission overdrive']: txtype = "Automatic"
        elif txtype.lower() in ['m/t','mt']: txtype = "Manual"
        elif txtype.lower()=="variable": txtype="CVT"
        elif txtype=="Transmission w/Dual Shift Mode": txtype="Dual Shift"
 
    transmission_type.append(txtype)
    
cdf["gears"] = gear
cdf["transmission_type"] = transmission_type

In [ ]:
cdf.gears.unique()

In [ ]:
cdf.transmission_type.unique()

In [ ]:
cdf.engine.sample(5)

In [ ]:
def extract_engine_data(engine_info):
    pattern = re.search(r'(\d{1,4}.\d{1,2}HP)?\s?(\d{1,2}.\d{1,2}[L|\sLiter])?[A-Za-z\s]{0,}(\d{1,2})?\s?[Cylinder]{0,}',engine_info,re.IGNORECASE)
    
    hp = pattern.group(1) if pattern.group(1) else None
    capacity = pattern.group(2) if pattern.group(2) else None
    cylinders = pattern.group(3) if pattern.group(3) else None
    return hp, capacity, cylinders

horsepower, capacity, cylinders = [],[],[]
i=0
for engine in cdf.engine:
    hp, cap, cy = extract_engine_data(engine)
    if hp!=None:
        hp = float(hp.lower().split("hp")[0]) 

    if cap!=None:
        cap = float(cap.lower().split("l")[0])    
    
    if cy!=None:
        cy = float(cy.split(" ")[0])
    
    horsepower.append(hp)
    capacity.append(cap)
    cylinders.append(cy)
    
    
cdf["horsepower"] = horsepower
cdf["engine_capacity"] = capacity
cdf["cylinders"] = cylinders

In [ ]:
cdf[['engine','horsepower','engine_capacity','cylinders']].sample(10)

In [ ]:
cdf.ext_color.unique()

In [ ]:
cdf.ext_color.value_counts(normalize=True).head(15)

In [ ]:
cdf.int_color.value_counts(normalize=True).head(15)

In [ ]:
base_colors = ['white','black','grey','gray','blue','red','yellow','silver','green','beige','gold','orange','brown','ebony','purple']

In [ ]:
def find_base_color(text):
        for color in base_colors:
            if color in text:
                return color
        return "uncommon"  
    
    
cdf.ext_color = cdf.ext_color.apply(lambda x: x.lower() if isinstance(x, str) else x)
cdf.int_color = cdf.int_color.apply(lambda x: x.lower() if isinstance(x, str) else x)
cdf.ext_color = cdf.ext_color.apply(find_base_color)
cdf.int_color = cdf.int_color.apply(find_base_color)
cdf.ext_color = cdf.ext_color.replace({"grey":"gray"})
cdf.int_color = cdf.int_color.replace({"grey":"gray"})

In [ ]:
cdf.ext_color.unique()

In [ ]:
cdf.ext_color.value_counts(normalize=True)*100

In [ ]:
cdf.int_color.value_counts(normalize=True)*100

In [ ]:
cdf.sample(2)

In [ ]:
# def fill_clean_title(row):
#     if pd.isna(row['clean_title']):
#         if row['accident_reported'] == 'No':
#             return True
#         elif row['accident_reported'] == 'Yes':
#             return False
#     return row['clean_title']

# # Apply the function to each row
# df['clean_title'] = df.apply(fill_clean_title, axis=1)
# dt['clean_title'] = dt.apply(fill_clean_title, axis=1)